In [1]:
import pandas as pd
import os


In [ ]:
# Setup paths for local execution
import os
import sys

# Project directories
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
DATA_DIR = os.path.join(BASE_DIR, 'data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'outputs')
SRC_DIR = os.path.join(BASE_DIR, 'src')

# Add src to path
sys.path.insert(0, SRC_DIR)


Mounted at /content/drive/


In [16]:
deep_future_non_zero_mape = pd.read_csv('../../data/non-zero-mean_df.csv')

In [17]:
lgb_cluster_non_zero_mape = pd.read_csv('../../data/lgb_cluster_mape.csv')

In [18]:
lgb_non_zero_distance = pd.read_csv('../../data/lgbnon-zerointerval_mape.csv')

In [19]:
deep_future_non_zero_mape = deep_future_non_zero_mape.rename(columns={'error':'deep_future'})

In [20]:
lgb_cluster_non_zero_mape = lgb_cluster_non_zero_mape.rename(columns={'error':'lgb_cluster'})

In [21]:
lgb_non_zero_distance = lgb_non_zero_distance.rename(columns={'error':'lgb_non_zero_distance'})

In [23]:
mape_together = lgb_non_zero_distance.merge(lgb_cluster_non_zero_mape,on=['StockCode'],how='left')

In [40]:
mape_together = mape_together.merge(deep_future_non_zero_mape,on=['StockCode'],how='left')

In [41]:
mape_together.columns

Index(['StockCode', 'lgb_non_zero_distance', 'lgb_cluster', 'deep_future'], dtype='object')

In [43]:
mape_together['model'] = mape_together[['lgb_non_zero_distance', 'lgb_cluster', 'deep_future']].idxmin(axis=1)

In [44]:
lgb_cluster = pd.read_csv('../../outputs/lgb_clusterdistanceforecast.csv')

In [50]:
lgb_cluster.shape

(24192, 4)

In [46]:
lgb_non_zero_distance = pd.read_csv('../../outputs/lgb_zerodistanceforecast.csv')

In [47]:
deep_future = pd.read_csv('../../outputs/deep_future_forecast.csv')

In [48]:
lgb_cluster_cluster = lgb_cluster.loc[lgb_cluster['StockCode'].isin(mape_together.loc[mape_together['model']=='lgb_cluster','StockCode']),]

In [51]:
lgb_non_zero_stock = lgb_non_zero_distance.loc[lgb_non_zero_distance['StockCode'].isin(mape_together.loc[mape_together['model']=='lgb_non_zero_distance','StockCode']),]

In [52]:
deep_future_stock = deep_future.loc[deep_future['StockCode'].isin(mape_together.loc[mape_together['model']=='deep_future','StockCode']),]

In [53]:
combined_forecast = pd.concat([lgb_cluster_cluster,lgb_non_zero_stock,deep_future_stock],axis=0)

In [54]:
rest_out_forecast = lgb_cluster.loc[~lgb_cluster['StockCode'].isin(combined_forecast['StockCode']),]

In [56]:
final_forecast= pd.concat([combined_forecast,rest_out_forecast],axis=0)

In [57]:
final_forecast.to_csv('../../outputs/final_forecast.csv',index=False)